## **Cálculo de las emisiones de CO2 emitidas por la generación de electricidad en Florida**

### Esta API calcula la emisión de CO2 para algunas actividades comunes. Es de libre acceso

### Para este propósito se utilizará la REST-API que puede ser consulada en: 
#### [Carbon interface](https://docs.carboninterface.com/#/)


In [10]:
import requests
import pandas as pd
import json

In [95]:
key = "KDPebxwd6cjRYVYycS6vA"
status_codes={
    200:"OK",
    201:"Object created",
    400:"Bad request",
    401:"Unauthorized",
    403:"Forbidden",
    404:"Not found",
    422:"Unprocessable entity",
    429:"Too many request",
    500:"Internal Server Error",
    503:"Service Unavailable"
        }


In [91]:
def acceso(key):
    url="https://www.carboninterface.com/api/v1/auth"
    authorization=requests.get(url,headers={"Authorization": "Bearer " + key})
    if authorization.status_code == 200:
        return print("Connection authorized")     
    else:
        return print(f"Connection failed, status code: {authorization.status_code} \nMeaning: {status_codes[authorization.status_code]}")

In [117]:
acceso(key)

Connection authorized


In [141]:
def unidad_electricidad():

    unidad=input("Introduce las unidades en las que la energía será medida. Las opciones" +
                    " válidas son: \n- mwh \n-kwh\n\n"+
                    "Para salir, presione s")
    unidad=unidad.lower()
    
    if unidad == "s":
        print("Hasta luego")
        return 
        
    elif (unidad != "kwh" and unidad != "mwh"): 
        print("El valor introducido no coincide con ninguna opción válida, intente de nuevo.")
        return unidad_electricidad()
    

In [81]:
def valor_electricidad():
    try:
        valor=float(input("Introduce la cantidad de energía eléctrica utilizada:"))
        return valor
    except:
        print("El valor introducido no es válido, intente de nuevo")
        return valor_electricidad()

In [157]:
def electricidad():
    
    type="electricity"
    unidad=unidad_electricidad()
    valor=valor_electricidad()
    country="us"
    
    header={
        "Content-Type":"application/json",
        "Authorization": "Bearer " + key,
    }
    
    data={
        "type":type,
        "electricity_unit":str(unidad),
        "electricity_value":valor,
        "country":country,
        "state":"fl"
    }
    
    url="https://www.carboninterface.com/api/v1/estimates"
    r=requests.post(url,headers=header,data=json.dumps(data))
    d=pd.DataFrame(json.loads(r.text)) #return a DataFrame, but the first row we have 
    # the information we need
    return pd.json_normalize(d.loc[d.index[0]].data)
    


In [158]:
data = electricidad()
data

,country,state,electricity_unit,electricity_value,estimated_at,carbon_g,carbon_lb,carbon_kg,carbon_mt
0,us,fl,None,42.0,2023-05-11T00:07:48.896Z,16710476,36840.29,16710.48,16.71


In [159]:
print("En EE.UU. en el estado de Florida, la cantidad de CO2 emitida en toneladas son: ",
      data["carbon_mt"][0], 
      "\nla cual fue estimada en ",
      data["estimated_at"][0])

En EE.UU. en el estado de Florida, la cantidad de CO2 emitida en toneladas son:  16.71 
la cual fue estimada en  2023-05-11T00:07:48.896Z
